# CITS4404 Firefly Algorithm Trading Bot
#### by: Davinh Dang (22717235), Dayu Liu (24188516), Ethan Young (23450844), Flavian Jerotich (24001784), Kushan Jayasekera (24205163), Xin Wang (24201533)

# README
This folder contains this `ipynb` file to run bot training and trading, dataset is imported from kaggle.

# 1. Dataset Processing
This step involves processing kaggle dataset. Dates and Closing prices are extracted with their corresponding values, ready for use for further processing for the analysis.
### Get required Python Modules

In [6]:
# TODO Required modules
!pip install torch
!pip install numpy
!pip install pandas
!pip install kagglehub


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import libraries
# Essentials
import numpy as np

# Data Processing
import pandas as pd
import kagglehub

# FIXME Modeling
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import time


# 1. Data Processing

In [ ]:
# Download dataset from Kaggle
path = kagglehub.dataset_download("prasoonkottarathil/btcinusd")
print("Path to dataset files:", path)

fileName = "BTC-Daily.csv"  #! Other candidates include BTC-{2017-2021}min.csv, BTC-Hourly.csv (Which are hourly data and may not be suitable)

def preprocess_data(path, fileName):
    # Open dataset and load file as CSV
    file_path = f"{path}/{fileName}"
    data = pd.read_csv(file_path)

    # Extract relevant columns
    dates = data['date'].tolist()
    prices = data['close'].tolist()

    # Create a DataFrame
    df = pd.DataFrame({'date': dates, 'price': prices})
    return df

train_df = preprocess_data(path, fileName)

# Variables for model
dates = train_df['date']
prices = train_df['price']
print("Dates:", dates[:5])
print("Prices:", prices[:5])

Path to dataset files: C:\Users\Administrator\.cache\kagglehub\datasets\prasoonkottarathil\btcinusd\versions\4


# 2. Filter Implementation
implementing the SMA filter function

### SMA Filter Implementation 

In [ ]:
def pad(P, N):
    flip = np.flip(P[1:N]) # TODO The project instruction have a negate sign before the flip function, could be a typo
    return np.append(flip, P)

def sma_filter(P, N):
    # pad the series
    P_pad = pad(P, N)
    # Roll the window and calculate the weighted average
    sma_full = pd.Series(P_pad).rolling(window=N).mean().to_numpy()
    return sma_full[N-1:]

#! To generate an N-day SMA we can simply call
sma10 = sma_filter(prices, 10)
sma20 = sma_filter(prices, 20)
print("sma10:", sma10[20:25])

sma10: [42625.384 43034.841 43366.134 43218.01  42902.534]


# 2. Model Implementation (TODO)


### Model's Hyperparameters (TODO)

In [ ]:
# Variable Initialization
lb = 10 # Short-term moving average window size
ub = 40 # Long-term moving average window size

### Model Constructor (TODO)

In [ ]:
# FIXME from https://github.com/firefly-cpp/FireflyAlgorithm/blob/master/fireflyalgorithm/fireflyalgorithm.py
import numpy as np
from numpy.random import default_rng


class FireflyAlgorithm:
    def __init__(self, pop_size=20, alpha=1.0, betamin=1.0, gamma=0.01, seed=None):
        self.pop_size = pop_size
        self.alpha = alpha
        self.betamin = betamin
        self.gamma = gamma
        self.rng = default_rng(seed)

    def run(self, function, dim, lb, ub, max_evals):
        fireflies = self.rng.uniform(lb, ub, (self.pop_size, dim))
        intensity = np.apply_along_axis(function, 1, fireflies)
        best = np.min(intensity)

        evaluations = self.pop_size
        new_alpha = self.alpha
        search_range = ub - lb

        while evaluations <= max_evals:
            new_alpha *= 0.97
            for i in range(self.pop_size):
                for j in range(self.pop_size):
                    if intensity[i] >= intensity[j]:
                        r = np.sum(np.square(fireflies[i] - fireflies[j]), axis=-1)
                        beta = self.betamin * np.exp(-self.gamma * r)
                        steps = new_alpha * (self.rng.random(dim) - 0.5) * search_range
                        fireflies[i] += beta * (fireflies[j] - fireflies[i]) + steps
                        fireflies[i] = np.clip(fireflies[i], lb, ub)
                        intensity[i] = function(fireflies[i])
                        evaluations += 1
                        best = min(intensity[i], best)
        return best

### Model Training (TODO)

In [ ]:
# Problem function to run
# FIXME from https://github.com/firefly-cpp/FireflyAlgorithm/blob/master/fireflyalgorithm/problems.py
def sphere(x):
    return np.sum(x**2)

# FIXME from https://github.com/firefly-cpp/FireflyAlgorithm/blob/master/examples/run.py
FA = FireflyAlgorithm()
best = FA.run(function=sphere, dim=10, lb=lb, ub=ub, max_evals=10000)
print(best)

0.6606683520227736


# 3. Trading strategy (TODO)

In [ ]:
# FIXME from ChatGPT
def simulate_trading(prices, SMA1, SMA2, initial_capital=1000):
    sma1 = sma_filter(SMA1)
    sma2 = sma_filter(SMA2)

    signal = np.where(sma1 > sma2, 1, 0)  # 1 = Buy, 0 = Sell
    position = 0
    capital = initial_capital
    shares = 0

    for i in range(1, len(prices)):
        price_today = prices.iloc[i]
        signal_today = signal[i]

        # Buy signal (if not holding)
        if signal_today == 1 and position == 0:
            shares = capital / price_today
            capital = 0
            position = 1
        # Sell signal (if holding)
        elif signal_today == 0 and position == 1:
            capital = shares * price_today
            shares = 0
            position = 0

    # Liquidate at end
    if position == 1:
        capital = shares * prices.iloc[-1]

    return capital


In [ ]:
# Assume you already got these from Firefly output:
best_sma1, best_sma2 = 10, 20

final_value = simulate_trading(prices, best_sma1, best_sma2)
print(f"Final capital after simulation: ${final_value:.2f}")